In [1]:
import pandas as pd
from recbole.quick_start.quick_start import load_data_and_model
from recbole.utils.case_study import *
import os

In [3]:
import pandas as pd
from recbole.quick_start.quick_start import load_data_and_model
from recbole.utils.case_study import *
import os

k = 100

for platform in ["DEEZER"]:
    for country in ["FR", "DE", "BR"]:
        for model_name in ["ItemKNN", "NeuMF"]:

# for platform in ["LFM"]:
#     for country in ["BR"]:
#         for model_name in ["NeuMF"]:

            # load prediction data, interactions and users
            # print(platform, country, model_name)
            saved_models_filenames = os.listdir(
                "saved/" + platform + "/" + country + "/" + model_name
            )
            for file_name in saved_models_filenames:
                print(platform, country, model_name, file_name)
                config, model, dataset, train_data, valid_data, test_data = (
                    load_data_and_model(
                        model_file="saved/"
                        + platform
                        + "/"
                        + country
                        + "/"
                        + model_name
                        + "/"
                        + file_name,
                    )
                )
                inter = pd.read_csv(
                    "dataset/"
                    + platform
                    + "_"
                    + country
                    + "/"
                    + platform
                    + "_"
                    + country
                    + ".inter"
                )
                users = inter["user_id:token"].unique().tolist()

                # for each user, get top k predictions
                res = []
                for i, u in enumerate(users):
                    uid_series = dataset.token2id(dataset.uid_field, [str(u)])
                    print(uid_series)
                    topk_score, topk_iid_list = full_sort_topk(
                        uid_series, model, test_data, k=k, device=config["device"]
                    )
                    # get the item id corresponding to the recbole item id
                    external_item_list = dataset.id2token(
                        dataset.iid_field, topk_iid_list.cpu()
                    )
                    for j in range(0, k):
                        res.append(
                            [
                                u,
                                external_item_list[0][j].item(),
                                topk_score[0][j].item(),
                            ]
                        )


                df = pd.DataFrame(res, columns=["user_id", "media_id", "score"])
                df.to_csv(
                    "predicted/"
                    + platform
                    + "/"
                    + country
                    + "/"
                    + model_name
                    + "/"
                    + file_name[:-4]
                    + ".csv"
                )


DEEZER FR ItemKNN Apr-17-2024_13-15-58.pth


23 Apr 16:46    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 318
state = INFO
reproducibility = False
data_path = dataset/DEEZER_FR
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10, 100]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parame

KeyboardInterrupt: 